# AlexNet(Theory)

[AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)是2012年Imagenet竞赛的冠军模型.

模型结构为:

![](../../../picture/180.png)

Ng给出的模型为:

![](../../../picture/181.png)

可以发现,Ng给出的模型和原论文的模型的Input是不一样的,实际上是原论文出现了出错(当然这个错误除了问Alex没人知道),Ng的Input输入是正确的,因为如果输入是$224,F=11,S=4$是产生不了$(55,55)$的.在cns231n中提到可能是由于Alex忘记了padd=3了.另外在原论文中,在$Pool1$和$Pool2$后是存在LRN(Local Response Normalization)局部响应归一化的,后面我们再来看看为什么Ng中的Alex没有LRN.还有一点是Alex在训练的时候使用的是2块GPU进行训练,并且含有一块进行数据交互.主要是受到当时的GPU显存计算的限制.

下面我们以$Input=227$来详细看看Alex.

![](../../../picture/182.png)

**AlexNet**由八个模块组成.

**第一个模块(CONV1):**
![](../../../picture/183.png)

1) 输入$(227\times 227\times 3)$,$F=11,S=1$,输出$(55\times 55\times 96)$.

2) ReLu1后:$(55\times 55\times 96)$

3) MaxPool1:$F=3,S=2$,输出$(27\times 27\times 96)$

4) LRN1: local_size=$\frac{n}{2}$ = 5,输出$(27\times 27\times 96)$

在AlexNet中,其使用ReLu激活函数,也就是说使用ReLu激活函数取代了LeNet-5中的Sigmoid和Tanh激活函数,另外Alex在池化层中使用了MaxPool而非LeNet-5中的AveragePool.

并且在AlexNet中使用了LRN(Local Response Normalization)局部响应归一化,LRN模拟神经生物学上一个叫做"侧抑制(lateral inhibitio)"的功能,侧抑制指的是被激活的神经元会抑制相邻的神经元.LRN局部响应归一化借鉴侧抑制的思想实现局部抑制,使得**使得其中响应比较大的值变得相对更大,并抑制其他反馈较小的神经元**进而提高了模型的泛化能力.需要注意的是:**LRN只对数据相邻区域做归一化处理,不改变数据的大小和维度**.

**Ps:**

> LRN(Local Response Normalization)

> 在paper中给出如下的公式定义:

> ![](../../../picture/184.png)

> 公式中$a$表示输入,也就是经过卷积层和池化层的结果$(batch,heights,widths,channels)$,$a^{i}_{x,y}$表示对于第$a$个特征图的第$i$个卷积核结构中的一个位置$[a,b,c,i]$,可以理解为在输入第$a$个特征图的第$i$个通道下的高度为$b$,宽度为$c$的点.

> ![](../../../picture/185.png)

> 在论文中$N$表示总的卷积核个数,$a,\frac{n}{2},k,\alpha,\beta$分别为$input,depth\_radius,bias,alpha,beta$,其中$\frac{n}{2},k,\alpha,\beta$都是超级参数,在论文中给出的建议是$k=2,n=5,\alpha=10^{-4},\beta=0.75$.特别需要注意的是,公式中的$\sum$叠加方向是沿着通道方向的,也就是一个点同方向的前面$\frac{n}{2}$个通道(最小为0)和后面$\frac{n}{2}$个通道(最大为N-1)的点的平方和.示意图如下:

> ![](../../../picture/186.png)

我们可以使用Tensorflow中的代码```tf.nn.lrn(input,depth_radius=None,bias=None,alpha=None,beta=None,name=None)```来演示一下:

In [4]:
import tensorflow as tf
import numpy as np

x = np.array([i for i in range(1,33)]).reshape([2,2,2,4]) # (batch=2,height=2,width=2,kernel_size=4)
y = tf.nn.lrn(input=x,depth_radius=2,bias=0,alpha=1,beta=1)
 
with tf.Session() as sess:
    print(x)
    print('---------------')
    print(y.eval())

[[[[ 1  2  3  4]
   [ 5  6  7  8]]

  [[ 9 10 11 12]
   [13 14 15 16]]]


 [[[17 18 19 20]
   [21 22 23 24]]

  [[25 26 27 28]
   [29 30 31 32]]]]
---------------
[[[[0.07142857 0.06666667 0.10000001 0.13793103]
   [0.04545454 0.03448276 0.04022989 0.05369128]]

  [[0.02980132 0.02242153 0.02466368 0.03287672]
   [0.0220339  0.01654846 0.0177305  0.02363368]]]


 [[[0.0174538  0.01310044 0.01382824 0.01843318]
   [0.01444292 0.01083744 0.01133005 0.01510384]]

  [[0.01231527 0.00923952 0.00959488 0.01279123]
   [0.01073279 0.00805153 0.00831991 0.01109185]]]]


我们将矩阵转换成图片的格式:

![](../../../picture/187.png)

比如我们要取26,按照我们的设定```depth_radius=2,bias=0,alpha=1,beta=1```,那么实际上$i=1$,则$\underset{j=min(0,1-2)}{\overset{max(4-1,1+2)}{\sum}}$,同时因为$n=4$,那么实际上就会有$n=4$个点:

$26/(0+1*(25^{2}+26^{2}+27^{2}+28^{2}))^{1}=0.00923952$

以上来摘取自[深度学习的局部响应归一化LRN(Local Response Normalization)理解](https://blog.csdn.net/yangdashi888/article/details/77918311),但是博客中说总共有$n+1$个点是不对的,因为某一个点肯定是属于前$\frac{n}{2}$或者后$\frac{n}{2}$通道中的之一,所以总共应该有$n$个点,从例子中同样也能看出.

LRN概念是在AlexNet模型中首次提出,在GoogLeNet中也有应用,但是LRN的实际作用存在争议,如在2015年[Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)论文中指出LRN基本没什么用.

AlexNet还应用了Overlapping(重叠池化),重叠池化就是池化操作在部分像素上有重合.池化核大小是$n\times n$,步长是$k$如果$k=n$,则是正常池化,如果 $k<n$, 则是重叠池化.论文中说,重叠池化的运用减少了$top-5$和$top-1$错误率的$0.4\%$和$0.3\%$.重叠池化有避免过拟合的作用.


![](../../../picture/188.png)

**第二个模块(CONV2):**

![](../../../picture/189.png)

1) 输入:$(27\times 27\times 96),F=5,S=1$,输出$(27\times 27\times 256)$,输入与输出大小不变,所以是做了Padd="SAME".

2) ReLu2: $(27\times 27\times 256)$

3) Pool2: $F=3,S=2$,输出$(13\times 13\times 256)$,$(\frac{27-13}{2}+1=13)$

4) LRN2: local_size=$\frac{n}{2}$ = 5,输出$(13\times 13\times 256)$

在AlexNet的conv2中使用了same padding,保持了卷积后图像的宽高不缩小.

**Ps:**

- 由于Alex当时是使用的2块GPU计算,所以Conv2的group=2,也就是相当于有并排的两个卷积层,每个卷积层计算输入通道的一半,并且产生的输出是输出通道的一半,随后将这两个输出连接起来.一般没有多GPU训练时group=1.

**第三,四个模块(CONV3,CONV4):**

![](../../../picture/190.png)

**CONV3**

1) 输入:$(13\times 13\times 256),F=3,S=1$,输出$(13\times 13\times 384)$,输入与输出大小不变,所以是做了Padd="SAME".

2) ReLu3: $(13\times 13\times 384)$

3) 没有Pool层和LRN层.

**CONV4**

1) 输入:$(13\times 13\times 384),F=3,S=1$,输出$(13\times 13\times 384)$,输入与输出大小不变,所以是做了Padd="SAME",且深度也没有发生变化.

2) ReLu4: $(13\times 13\times 384)$

3) 没有Pool层和LRN层.


**第五个模块(CONV5):**

![](../../../picture/191.png)

**CONV5**

1) 输入:$(13\times 13\times 384),F=3,S=1$,输出$(13\times 13\times 256)$,输入与输出大小不变,所以是做了Padd="SAME".

2) ReLu5:$(13\times 13\times 256)$

3) Pool5:$F=3,S=2$,输出$(6\times 6\times 256)$($\frac{13-3}{2}+1=6$)

3) 有Pool层和没有LRN层.

**第六个模块:**
![](../../../picture/192.png)

**FC6**

1) 输入:$(6*6*256)$,输出$(4096,1)$

2) ReLu6: $(4096,1)$

3) dropout6:$(4096,1)$

4) 最后输出:$(4096,1)$

AlexNet在fc6全连接层引入了drop out的功能.dropout是指在深度学习网络的训练过程中,对于神经网络单元,按照一定的概率将其暂时从网络中丢弃(保留其权值),不再对前向和反向传输的数据响应.注意是暂时,对于随机梯度下降来说,由于是随机丢弃,故而相当于每一个mini-batch都在**训练不同的网络**,drop out可以有效防止模型过拟合,让网络泛化能力更强,同时由于减少了网络复杂度,加快了运算速度.还有一种观点认为drop out有效的原因是对样本增加来噪声,变相增加了训练样本.

**第七,八个模块:**
![](../../../picture/193.png)

**FC7**

1) 输入:$(4096,1)$,输出$(4096,1)$

2) ReLu6: $(4096,1)$

3) dropout6:$(4096,1)$

4) 最后输出:$(4096,1)$

**FC8**

1) 输入:$(4096,1)$,输出$(1000,1)$

FC8输出一千种分类的概率.


**Ps:**

- 在AlexNet中Dropout设置为0.5.

- 使用SGD OPtimizer.

- 在Backward中对于MaxPool 使用的是upsampling中的插值方式,一般upsampling有三种方式:

    - bilinear:线性插值.
    
    - Deconvolution:或者叫转置卷积.
    
    - unpooling: 上池化(由ZFNet)网络提出
    
    - 详情查看[计算机视觉中upsampling(上采样)的三种方式](https://blog.csdn.net/u014451076/article/details/79156967)

### AlexNet 特点:

(1) 成功使用了Relu激活函数,成功解决了sigmoid在网络较深时[梯度弥散](https://www.cnblogs.com/yangmang/p/7477802.html)的问题,加快了训练速度,虽然ReLu函数很早就被提出,但是在AlexNet中才被很好的证实.

(2) 使用Dropout随机关闭一些神经元,进而避免过拟合.

(3) 使用最大池化,从而避免特征图模糊,并且AlexNet中提出Overlapping,这样会使得输出之间会有重叠和覆盖,提升了特征的丰富性.

(4) 提出了LRN层,对局部神经元的活动创建竞争机制,使得其中响应比较大的值变得相对更大,并抑制其他反馈较小的神经元,提升了模型的泛华能力.

(5) 使用了多GPU(虽然是为了解决当时GPU运算能力).

(6) **数据增强.**

> 为了防止过拟合论文进行了数据增广, 将原始图像进行平移, 水平翻转等变换

>![](../../../picture/194.png)

>原始图像为一个大图$a$,先把一短边缩小到$256$维得到$b$,然后在$b$的中心取$(256,256)$的正方形图片得到$c$,然后在$c$上随机提取$(256-224=32)224∗224$的小图片作为训练样本,然后在结合图像水平反转($\times 2$)来增加样本达到数据增益.这种增益方法是样本增加了$2048(32*32*2)$倍.

> 对图片的RGB通道进行强度改变,在训练集的RGB通道上做[PCA](https://www.cnblogs.com/pinard/p/6239403.html),但是不降维,只取特征向量和特征值,对训练集上每张图片的每个像素$I_{x,y}=[I^{R}_{x,y},I^{G}_{x,y},I^{B}_{x,y}]$加上下面的值形成新的数据:

> $[p1,p2,p3][\alpha_{1}\lambda_{1},\alpha_{2}\lambda_{2},\alpha_{3}\lambda_{3}]^{T}$

> 其中$p_{i}$表示特征向量,$\lambda_{i}$表示特征值,$\alpha_{i}$表示均值为0,方差为1的高斯随机变量

> 在沦为中指出top-1误差降低了$1%$

> 在测试的时候,输入$(256,256)$的图片,获取四个角和中心提取5个$(224,224)$片段,在水平翻转后形成10个样本,继而求这10个样本的最大概率值.

**结果:**

<img src="../../../picture/195.png" height=300 width=300>

该图是第一层卷积后的结果,上半部分为GPU1学习到的,下半部分由GPU2学习到的.

GPU1卷积核可视图可以看出,卷积核具有方向,"频率"选择性,还有一些圆点状的,也有许多死的卷积核;

GPU2学习到的特征大部分体现了颜色特征,可以看出不同卷积核对于同一输入学习到的内容也是不一样的.

另一个值得注意的地方就是在最后的一个具有4096个神经元的全连接层,通过计算两个样本的欧氏距离,欧氏距离比较近的图片具有相似的图片对象.说明网络确实提取到了有用的特征:

<img src="../../../picture/196.png" height=300 width=300>

也就是说,如果是相似的图片,那么最后一个4096全连接层学习到的特征就应该是"很近"的,也就说明了网络确实学到了有用的特征,但是如果直接使用4096的全连接层计算欧式距离,那么效率是低下的,所以Alex通过一个[Auto-encoder](https://morvanzhou.github.io/tutorials/machine-learning/ML-intro/2-5-autoencoder/)算法,来生产一个维数较少的二元向量,这样就可以通过比较这个维数较小的向量来进行图像检索.



**Reference:**

[1] [深度学习——卷积神经网络的经典网络](https://www.cnblogs.com/guoyaohua/p/8534077.html)

[2] [AlexNet神经网络结构](https://blog.csdn.net/dcrmg/article/details/79241211)

[3] [AlexNet简说](http://simtalk.cn/2016/09/20/AlexNet/)